In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import *

class Set:

    @abstractmethod
    def upper_half(self, axis: int, value: float) -> Set: pass
    
    @abstractmethod
    def lower_half(self, axis: int, value: float) -> Set: pass


_SetGenerator = Callable[[Set], Set]

@dataclass
class _Var:
    idx: int
    name: str

    def __lt__(self, rhs: float) -> _SetGenerator:
        return lambda s: s.lower_half(self.idx, rhs)
    
    def __gt__(self, rhs: float) -> _SetGenerator:
        return lambda s: s.upper_half(self.idx, rhs)

@dataclass
class Space(ABC):

    names: list[str]
    approx: str = 'exact'
    _generator: _SetGenerator = lambda s: s

    @property
    def ndim(self) -> int:
        return len(self.names)

    @property
    def vars(self) -> list[_Var]:
        return [_Var(self, name, i) for i, name in enumerate(self.names)]

    @property
    def subspace(self) -> Space:
        return Space(self.names, 
                     self.approx,
                     self._generator)

    def __getitem__(self, name: str) -> _Var:
        if name not in self.names:
            return super().__getitem__(name)
        else:
            return _Var(self, name, self.names.index(name))
        
    def __call__(self, s: Set) -> Set:
        return self._generator(s)

    def __or__(self, rhs: _SetGenerator | tuple) -> Space:
        if isinstance(rhs, tuple):
            return rhs[0](self) | rhs[1:] if rhs else self
        elif isinstance(rhs, _SetGenerator):
            s = self.subspace
            s._generator = lambda s: rhs(self(s))
            return s
        else:
            return NotImplemented


In [ ]:
S = Space(*'x y a v'.split())
X, Y, A, V = S.vars

s = S.subspace
s = s | X > 3, Y < 2

# s: lower 2 (upper 3 (ident s))

In [ ]:

class Solver(ABC):

    @abstractmethod
    def complement(self, a: Set) -> Set: pass

    @abstractmethod
    def intersect(self, a: Set) -> Set: pass
    
    @abstractmethod
    def reach(self, a: Set, b: Set) -> Set: pass
    
    @abstractmethod
    def rci(self, a: Set) -> Set: pass
